In [ ]:
# Standard library imports
import sys

# Related third party imports
from tqdm import tqdm
import pandas as pd

# Local application/library specific imports.
# paradoxcell
sys.path.append('/10Gdata/sjt')
import paradoxcell as pc


In [ ]:
pmn_reaction_directory = '/10Gdata/sjt/data/database/pmn'
pmn_compound_directory = '/10Gdata/sjt/data/database/pmn'

pmn_reaction = pc.DataFile(
    pmn_reaction_directory,
    'reactions.dat',
    'cp1252',
    '\\'
)

pmn_reaction_paths = pmn_reaction.get_path()

pmn_compound = pc.DataFile(
    pmn_compound_directory ,
    'compounds.dat',
    'cp1252',
    '\\'
)

pmn_compound_paths = pmn_compound.get_path()

In [ ]:
df_pmn_reaction = pd.DataFrame()

for path in tqdm(pmn_reaction_paths):
    with open(path,encoding='cp1252') as f:
        data = f.read()
    list_pmn_reaction = list()
    organism = "".join([z.split(':')[1].strip() for z in data.split('//\n')[0].split('#') if z.startswith(' Organism: ')])
    for i in data.split('//\n'):
        ec = ",".join([t for k in i.split('\n') if k.startswith('EC-NUMBER') for t in k.split(' - ')[1:]])
        id = ",".join([t for k in i.split('\n') if k.startswith('UNIQUE-ID') for t in k.split(' - ')[1:]])
        left = ",".join([t for k in i.split('\n') if k.startswith('LEFT') for t in k.split(' - ')[1:]])
        right = ",".join([t for k in i.split('\n') if k.startswith('RIGHT') for t in k.split(' - ')[1:]])
        balance = ",".join([t for k in i.split('\n') if k.startswith('REACTION-BALANCE-STATUS') for t in k.split(' - ')[1:]])
        direction = ",".join([t for k in i.split('\n') if k.startswith('REACTION-DIRECTION') for t in k.split(' - ')[1:]])
        list_pmn_reaction.append((organism,ec,id,left,right,balance,direction))
    df = pd.DataFrame(list_pmn_reaction,columns=['organism','ec','id','left','right','balance','direction'])
    df_pmn_reaction = pd.concat([df_pmn_reaction,df],ignore_index=True)

df_pmn_reaction

In [ ]:
df_pmn_compound = pd.DataFrame()

for path in tqdm(pmn_compound_paths):
    with open(path,encoding='cp1252') as f:
        data = f.read()
    list_pmn_compound = list()
    organism = "".join([z.split(':')[1].strip() for z in data.split('//\n')[0].split('#') if z.startswith(' Organism: ')])
    for i in data.split('//\n'):
        id = ",".join([t for k in i.split('\n') if k.startswith('UNIQUE-ID') for t in k.split(' - ')[1:]])
        dblink = ",".join([":".join([z.strip('(').strip() for z in t.split('"')[0:2]]) for k in i.split('\n') if k.startswith('DBLINKS') for t in k.split(' - ')[1:]])
        smiles = ",".join([t for k in i.split('\n') if k.startswith('SMILES') for t in k.split(' - ')[1:]])
        list_pmn_compound.append((organism,id,dblink,smiles))
    df = pd.DataFrame(list_pmn_compound,columns=['organism','id','dblink','smiles'])
    df_pmn_compound = pd.concat([df_pmn_compound,df],ignore_index=True)

df_pmn_compound['LIGAND-CPD'] = df_pmn_compound['dblink'].apply(lambda x : x[x.find('LIGAND-CPD')+11:x.find('LIGAND-CPD')+17] if x.find('LIGAND-CPD:') != -1 else None)

df_pmn_compound